In [68]:
import tensorflow as tf
import numpy as np

In [49]:
from moviepy.editor import AudioFileClip
from pathlib import Path

In [2]:
root_dir = Path('/vol/atlas/homes/gt108/db/CACAC')

In [40]:
with open((root_dir / 'labels.txt').as_posix()) as f:
    raw_labels = [x.strip().split(';') for x in f.readlines()]
    
    labels = {}
    for name, portion, g in raw_labels:
        labels.setdefault(portion, []).append((name, g == 'CDS'))

In [119]:
def get_audio(wav_file):
    file = AudioFileClip(str(root_dir / 'wav' / wav_file))
    file = file.set_fps(16000)
    audio = np.array(list(file.iter_frames()))[:, 0]
    audio = np.pad(audio, (0, 640 - audio.shape[0] % 640), 'constant')
    audio = audio.reshape(-1, 640)
    return audio

def _int_feauture(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feauture(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def serialize_sample(writer, sample_data):
    for i, (wav_file, label) in enumerate(sample_data):

        audio = get_audio(wav_file)
        example = tf.train.Example(features=tf.train.Features(feature={
                    'label': _int_feauture(label),
                    'raw_audio': _bytes_feauture(audio.tobytes()),
                }))

        writer.write(example.SerializeToString())
        del audio, label

In [129]:
len(labels['test'])

3594

In [122]:
for portion in labels:
    print(portion)
    
    writer = tf.python_io.TFRecordWriter(
        (root_dir / 'tf_records' / '{}.tfrecords'.format(portion)
    ).as_posix())
    
    serialize_sample(writer, labels[portion])

test
train
devel
